<center><h1>RFM Analysis</h1></center>

<hr/>

<p>Let's conduct an RFM Analysis using a retail data. It contains customer level data on transactions by date. It also got response information to a promotion campaign conducted by the organization.</p>

In [14]:
# Import all the Important Modules  
import pandas as pd
import matplotlib.pyplot as plt

# Supress 'Deprecated' Warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Read 'Sample_Order' CSV and store it in 'orders_df'
orders_df = pd.read_csv('/Users/jaypatel/Documents/University Material/Semester 8/IBM Project Code Files/RFM Analysis/data/Sample_Order.csv')

In [5]:
# Print few entries of 'orders_df'
orders_df.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franz�sisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


### Exploratory Data Analysis (EDA)

In [6]:
# Fetch 'Information' of Each Field
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   order_date   5009 non-null   object
 1   order_id     5009 non-null   object
 2   customer     5009 non-null   object
 3   grand_total  5009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 156.7+ KB


In [7]:
# Find Does there is any 'Null' OR 'Missing' Values
orders_df.isnull().sum()

order_date     0
order_id       0
customer       0
grand_total    0
dtype: int64

### Creation of RFM Table

In [17]:
# Initialize Time
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [18]:
# Make the date_placed column datetime
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

In [19]:
# Implementation of 'Aggregation' Function for BIffurcation
rfmTable = orders_df.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),                       # Frequency
                                        'grand_total': lambda x: x.sum()})                  # Monetary Value

# Type Conversion as 'INT'
rfmTable['order_date'] = rfmTable['order_date'].astype(int)

# Renaem all the 'Columns'
rfmTable.rename(columns={'order_date': 'recency', 
                        'order_id': 'frequency', 
                        'grand_total': 'monetary_value'}, inplace=True)

### Validation of the RFM Table

In [20]:
# Print Few Entries of 'rfmtable'
rfmTable.head()

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


In [21]:
# Find Entry of 'Aaron Hawkins'
aaron = orders_df[orders_df['customer']=='Aaron Hawkins']

# Verify all the Data of 'aaron'
aaron

,order_date,order_id,customer,grand_total
174,2011-05-13,CA-2011-113768,Aaron Hawkins,287
263,2011-04-22,CA-2011-122070,Aaron Hawkins,258
674,2011-12-31,CA-2011-157644,Aaron Hawkins,54
1188,2012-12-27,CA-2012-130113,Aaron Hawkins,991
2682,2013-03-21,CA-2013-162747,Aaron Hawkins,86
4084,2014-12-19,CA-2014-164000,Aaron Hawkins,19
4332,2011-10-25,US-2011-158400,Aaron Hawkins,49


#### What above Data Represents
Recency: 12 Days

Frequency: 7 Days

Monetary: 1744 $

### RFM Model for the Quantile Calculation

In [22]:
# Upper Quantile (Q3)= 0.75, Medium Quantile (Q2)= 0.5, Lower Quantile (Q1)= 0.25
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

# Print Few entries of 'quantiles'
quantiles

,recency,frequency,monetary_value
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


In [23]:
# Conversion of 'quantile' into dictionary. So, we can access 'Unique' Entries
quantiles = quantiles.to_dict()

# Print Few entries of 'quantiles'
quantiles

{'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0},
 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}

#### RFM Model Customer Segmentation

In [24]:
# Initialization of 'rfmSegmentation' Variable
rfmSegmentation = rfmTable

In [25]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [26]:
# Calculate Score of Individual Component in the Scale of 4
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [27]:
# Initialize of 'RFM' Class
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [28]:
# Store a Resultant Format using 'CSV' Format
rfmSegmentation.to_clipboard()
rfmSegmentation.to_csv('/Users/jaypatel/Documents/University Material/Semester 8/IBM Project Code Files/RFM Analysis/data/rfm-table.csv', sep=',')

In [32]:
# Top Five Customers According to 'RFM Table' Data
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Sanjit Engle,9,11,12210,1,1,1,111
John Lee,21,11,9801,1,1,1,111
Pete Kriz,9,12,8647,1,1,1,111
Harry Marie,2,10,8237,1,1,1,111
Lena Creighton,16,12,7661,1,1,1,111


#### Conclusion:

<b>The Above Score is like highest quantile directly proportional to high 'RFM' Score which refers to 'Disloyal' Customers</b>

<p>Therefore we have start with the lowest quantile for the identification of 'Loyal' Customer with 'RFM Class' of 111. Thus this was the RFM Analysis of 'Daily Store Transaction' Data</p>

<a href= "https://github.com/joaolcorreia/RFM-analysis/blob/master/RFM%20Analysis.ipynb">Reference</a>